In [ ]:
# Import necessary libraries
import pandas as pd
from scripts.data_processing import aggregate_experience_metrics, get_top_bottom_frequent
from scripts.user_engagement import kmeans_clustering
from scripts.visualizations import plot_distribution
from scripts.db_connection import get_data_from_postgres




In [ ]:

# Define database configuration
db_config = {
    'host': 'localhost',
    'database': 'telecom_data',
    'user': 'postgres',
    'password': 'Lkj;Asdf!@'
}

# Define SQL query
query = "SELECT * FROM xdr_data"

# Get data from PostgreSQL
df = get_data_from_postgres(query, db_config)

In [ ]:

# Task 3.1 - Aggregate Experience Metrics
experience_metrics = aggregate_experience_metrics(df)
print(experience_metrics.head())



In [ ]:
# Task 3.2 - Top, Bottom, and Frequent Values for 'Avg Bearer TP DL (kbps)'
top_bottom_frequent = get_top_bottom_frequent(df, 'Avg Bearer TP DL (kbps)')
print(top_bottom_frequent)


In [ ]:

# Task 3.2 - Top, Bottom, and Frequent Values for 'Avg RTT DL (ms)'
top_bottom_frequent_rtt = get_top_bottom_frequent(df, 'Avg RTT DL (ms)')
print(top_bottom_frequent_rtt)



In [ ]:
# Task 3.3 - Distribution Per Handset
df['avg_throughput'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2
plot_distribution(df, 'Handset Type', 'avg_throughput')


In [ ]:

# Task 3.4 - K-Means Clustering
# Convert relevant columns to numeric types, replacing non-numeric values with NaN
df['Total DL (Bytes)'] = pd.to_numeric(df['Total DL (Bytes)'], errors='coerce')
df['Total UL (Bytes)'] = pd.to_numeric(df['Total UL (Bytes)'], errors='coerce')
df['Dur. (ms)'] = pd.to_numeric(df['Dur. (ms)'], errors='coerce')
df['Bearer Id'] = pd.to_numeric(df['Bearer Id'], errors='coerce')

# Create the 'Total Traffic (Bytes)' column
df['Total Traffic (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

# Handle missing values (if any), only apply to numeric columns
df.fillna(df.mean(numeric_only=True), inplace=True)

# Create the 'Total Traffic (Bytes)' column
df['Total Traffic (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

# Handle missing values (if any), only apply to numeric columns
df.fillna(df.mean(numeric_only=True), inplace=True)

# Apply clustering
clusters, kmeans_model = kmeans_clustering(df, k=3)

# Print cluster centers
print(kmeans_model.cluster_centers_)